## Colab Prep

Execute the following code cells to whenever you open/restart the notebook in Google Colab.

In [1]:
!pip install "polars[all]"

In [2]:
!wget https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/sample_data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!unzip ./sample_data.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!pip install more_polars

# Joining Tables with `dfply`

In [5]:
import polars as pl

## Example

The following tables were taken from the Wikipedia page for joins and will provide a small but sufficiently complicated example for introducing joins.

In [6]:
dept = pl.read_csv("./sample_data/department.csv")
dept

DeptID,DeptName
i64,str
31,"""Sales"""
33,"""Engineering"""
34,"""Clerical"""
35,"""Marketing"""


In [7]:
empl = pl.read_csv("./sample_data/employee.csv")
empl

LastName,EmployeeID,DeptID
str,i64,i64
"""Rafferty""",102,31
"""Jones""",105,33
"""Heisenberg""",106,33
"""Robinson""",108,34
"""Smith""",110,34
"""Williams""",111,null


In [8]:
salary = pl.read_csv("./sample_data/salary.csv")
salary

EmployeeID,Year,Salary
i64,i64,i64
102,2020,52000
102,2021,53000
102,2022,54000
105,2020,73000
105,2021,74000
…,…,…
110,2021,59000
110,2022,61000
111,2020,72000


## LEFT JOIN

In [9]:
empl.join(dept, on='DeptID', how='left')

LastName,EmployeeID,DeptID,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""
"""Williams""",111,null,null


## When the index names differ

In [10]:
empl2 = empl.rename({'DeptID':'dept_id'})
empl2

LastName,EmployeeID,dept_id
str,i64,i64
"""Rafferty""",102,31
"""Jones""",105,33
"""Heisenberg""",106,33
"""Robinson""",108,34
"""Smith""",110,34
"""Williams""",111,null


In [11]:
empl2.join(dept, left_on='dept_id', right_on='DeptID', how='left')

LastName,EmployeeID,dept_id,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""
"""Williams""",111,null,null


## No RIGHT JOIN, use a LEFT JOIN

In [12]:
dept.join(empl, on='DeptID', how='left')

DeptID,DeptName,LastName,EmployeeID
i64,str,str,i64
31,"""Sales""","""Rafferty""",102
33,"""Engineering""","""Jones""",105
33,"""Engineering""","""Heisenberg""",106
34,"""Clerical""","""Robinson""",108
34,"""Clerical""","""Smith""",110
35,"""Marketing""",null,null


## Full Outer Join

In [13]:
empl.join(dept, on='DeptID', how='outer')

C:\Users\su6123ck\AppData\Local\Temp\ipykernel_2480\3270903518.py:1: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  empl.join(dept, on='DeptID', how='outer')


LastName,EmployeeID,DeptID,DeptID_right,DeptName
str,i64,i64,i64,str
"""Rafferty""",102,31,31,"""Sales"""
"""Jones""",105,33,33,"""Engineering"""
"""Heisenberg""",106,33,33,"""Engineering"""
"""Robinson""",108,34,34,"""Clerical"""
"""Smith""",110,34,34,"""Clerical"""
"""Williams""",111,null,null,null
null,null,null,35,"""Marketing"""


## Inner Join

In [14]:
empl.join(dept, on='DeptID', how='inner')

LastName,EmployeeID,DeptID,DeptName
str,i64,i64,str
"""Rafferty""",102,31,"""Sales"""
"""Jones""",105,33,"""Engineering"""
"""Heisenberg""",106,33,"""Engineering"""
"""Robinson""",108,34,"""Clerical"""
"""Smith""",110,34,"""Clerical"""


## Chaining multiple joins

To join more than two tables, simply dot-chain each join onto the first table.

```{python}
(table1
 .join(table2, ...)
 .join(table3, ...)
)
```

### Example - Average salary by department

Suppose we want to compute the average salary for each department, sorted from largest to smallest.  Before performing the query, make sure you understand the relationships between tables.

<img src="https://github.com/WSU-DataScience/dsci_325_module_7_more_data_management_in_python/raw/main/img/table_relationship.png" width="800">


#### Edit 1 - Joining the salary and employee tables

**Notes.**

1. We can't perform the aggregation until *after* joining the departments and salaries.
2. Since we are primarily interested in aggregating salaries, we need to use a *left* join with the salary table on the left.
3. To illustrate the coding process, we will show each successive edit in a separate cell.  In practice, all the code would be developed in one cell, but rerun to check the results at each step.

In [15]:
(salary
 .join(empl, on='EmployeeID', how='left')
).head()

EmployeeID,Year,Salary,LastName,DeptID
i64,i64,i64,str,i64
102,2020,52000,"""Rafferty""",31
102,2021,53000,"""Rafferty""",31
102,2022,54000,"""Rafferty""",31
105,2020,73000,"""Jones""",33
105,2021,74000,"""Jones""",33


#### Edit 2  - Joining on the departments

In [16]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
).head()

EmployeeID,Year,Salary,LastName,DeptID,DeptName
i64,i64,i64,str,i64,str
102,2020,52000,"""Rafferty""",31,"""Sales"""
102,2021,53000,"""Rafferty""",31,"""Sales"""
102,2022,54000,"""Rafferty""",31,"""Sales"""
105,2020,73000,"""Jones""",33,"""Engineering"""
105,2021,74000,"""Jones""",33,"""Engineering"""


#### Edit 3  - Group and aggregate

In [17]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .groupby('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
)

AttributeError: 'DataFrame' object has no attribute 'groupby'

#### Edit 4  - Sort the results from largest to smallest

In [ ]:
(salary
 .join(empl, on='EmployeeID', how='left')
 .join(dept, on='DeptID', how='left')
 .groupby('DeptName')
 .agg([pl.col('Salary').mean().alias('Average Salary'),
      ])
 .sort('Average Salary', reverse=True)
)

## Example 2 - Joining Batting to People

As an example of a joining on more than 1 column, consider the following task.

**Task:** Create a table with the total runs score for each team in 2009.  Include the teams proper name and the name of their home park.

In [ ]:
batting = pl.read_csv("./sample_data/baseball/core/Batting.csv")
batting.head()

In [ ]:
teams = pl.read_csv("./sample_data/baseball/core/Teams.csv")
teams.head()

## Things we need from `Batting.csv`

* Year
* Runs
* Team

In [ ]:
batting.head(2)

#### Selecting down to the necessary columns

In [ ]:
batting_select = (batting
                  .select(['teamID',
                           'yearID',
                           'R'])
                 )
batting_select.head()

In [ ]:
batting_select.shape

## Things we need from `Teams.csv`

* Year
* Runs
* Team

In [ ]:
teams.head(2)

#### Selecting down to the necessary columns

In [ ]:
teams_select = (teams.select(['yearID',
                              'teamID',
                              'name',
                              'park',
                             ]))
teams_select.head()

In [ ]:
teams_select.shape

## How can we match these columns?

We will need to match on both `teamID` and `yearID` because

1. Team names might change
2. The teams park depends on the year.

## Determining the correct join.

Facts of the task:

* We want batting statistics for all players in 2009
* We want only teams names and parks for teams that played that year.

**Question:** Which join? 
**Answer:** `batting >> left_join(teams)` on both year and 

## How to join on multiple columns

Pass a list of columns names to `on`

In [18]:
df_joined = batting_select.join(teams_select, on=['teamID', 'yearID'], how='left')
df_joined.head()

NameError: name 'batting_select' is not defined

In [19]:
df_joined.shape

NameError: name 'df_joined' is not defined

#### Filter, group by, and aggregate to get the desired result.

In [20]:
runs_per_team09 = (df_joined
                   .filter(pl.col('yearID') == 2009)
                   .groupby([pl.col('name'), pl.col('park')])
                   .agg(pl.col('R').sum().alias('total_runs')))
runs_per_team09.head()

NameError: name 'df_joined' is not defined

In [21]:
runs_per_team09.shape

NameError: name 'runs_per_team09' is not defined

## Joins are expensive!

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `inner_join` performs `min(len(A), len(B))` comparisons
* `outer_join` performs `len(A)*len(B)` row comparisons.
* `left_join` and `right_join` are somewhere inbetween (depending on how we treat multiples).

## Outer joins (can) produce LARGE tables.

**Be careful when joining large tables!! Especially for full outer joins!!!**

When joining table `A` to table `B`, 

* `outer_join` might produce `len(A)*len(B)` rows (worst case)
* `left_join` and `right_join` are similar if we don't drop multiples.

#### Last example 

In [22]:
len(batting_select)*len(teams_select)

NameError: name 'batting_select' is not defined

## The joining mantra: Filter/aggregate THEN join

By filtering and/or aggregating before joining, you will

* reduce the amount of work
* protect against really large output

#### Filter and aggregate to total runs per team per year

In [23]:
runs = (batting_select
        .filter(pl.col('yearID') == 2009)
        .groupby(pl.col('teamID'))
        .agg(pl.col('R').sum().alias('total_runs')))

runs.head()

NameError: name 'batting_select' is not defined

#### Filter down to 2009 teams

In [24]:
teams09 = (teams_select
           .filter(pl.col('yearID') == 2009)
          )
teams09.head()

NameError: name 'teams_select' is not defined

In [25]:
len(runs)*len(teams09)

NameError: name 'runs' is not defined

In [26]:
runs_per_team09_new = (runs 
                       .join(teams09, on='teamID', how='left')
                       .drop('teamID')
                      )
runs_per_team09_new.head()

NameError: name 'runs' is not defined

In [27]:
runs_per_team09_new.shape

NameError: name 'runs_per_team09_new' is not defined

## Don't worry about speed until it matters!

* I don't emphasize speed most of the time.
* In this case, either approach was nearly instant.
* Still ... `join`s are *so dangerous* that I make an exception here!

## <font color="red"> Exercise 7.3.1 </font>

Determine all the players that have hit more than 30 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the `Batting.csv`, `People.csv`, and `Teams.csv` files.  To get credit for this exercise, use the join methods presented above.

In [46]:
batting = pl.read_csv('./sample_data/baseball/core/Batting.csv')
batting

playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""","""NA""",1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""","""NA""",25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""","""NA""",29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""","""NA""",27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""","""NA""",25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""zimmejo02""",2019,1,"""DET""","""AL""",23,2,0,0,0,0,0,0,0,0,0,2,"""0""","""0""","""0""","""0""",0
"""zimmeky01""",2019,1,"""KCA""","""AL""",15,0,0,0,0,0,0,0,0,0,0,0,"""0""","""0""","""0""","""0""",0
"""zimmery01""",2019,1,"""WAS""","""NL""",52,171,20,44,9,0,6,27,0,0,17,39,"""0""","""0""","""0""","""2""",4


In [71]:
batting_prepared = (batting
                        .select(pl.col('playerID'), pl.col('yearID'), pl.col('teamID'), pl.col('HR'))
                        .filter(pl.col('HR')>30)
                   )
batting_prepared

playerID,yearID,teamID,HR
str,i64,str,i64
"""ruthba01""",1920,"""NYA""",54
"""ruthba01""",1921,"""NYA""",59
"""hornsro01""",1922,"""SLN""",42
"""ruthba01""",1922,"""NYA""",35
"""walketi01""",1922,"""PHA""",37
…,…,…,…
"""storytr01""",2019,"""COL""",35
"""suareeu01""",2019,"""CIN""",49
"""torregl01""",2019,"""NYA""",38


In [32]:
people = pl.read_csv('./sample_data/baseball/core/People.csv')
people.head()

playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""aardsda01""",1981,12,27,"""USA""","""CO""","""Denver""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""2015-08-23""","""aardd001""","""aardsda01"""
"""aaronha01""",1934,2,5,"""USA""","""AL""","""Mobile""",null,null,null,null,null,null,"""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""1976-10-03""","""aaroh101""","""aaronha01"""
"""aaronto01""",1939,8,5,"""USA""","""AL""","""Mobile""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""1971-09-26""","""aarot101""","""aaronto01"""
"""aasedo01""",1954,9,8,"""USA""","""CA""","""Orange""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""1990-10-03""","""aased001""","""aasedo01"""
"""abadan01""",1972,8,25,"""USA""","""FL""","""Palm Beach""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""2006-04-13""","""abada001""","""abadan01"""


In [82]:
people_prepared = (people
                       .with_columns(pl.concat_str(["nameFirst", "nameLast"], separator=" ").alias("playerName"))
                       .select(pl.col('playerID'), pl.col('playerName'))
                  )
people_prepared.head()

playerID,playerName
str,str
"""aardsda01""","""David Aardsma"""
"""aaronha01""","""Hank Aaron"""
"""aaronto01""","""Tommie Aaron"""
"""aasedo01""","""Don Aase"""
"""abadan01""","""Andy Abad"""


In [83]:
teams = pl.read_csv('./sample_data/baseball/core/Teams.csv')
teams.head()

yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,DivWin,WCWin,LgWin,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,CS,HBP,SF,RA,ER,ERA,CG,SHO,SV,IPouts,HA,HRA,BBA,SOA,E,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,str,str,i64,i64,str,str,str
1871,"""NA""","""BS1""","""BNA""",null,3,31,null,20,10,null,null,"""N""",null,401,1372,426,70,37,3,60,19,73,16,null,null,303,109,3.55,22,1,3,828,367,2,42,23,243,24,0.834,"""Boston Red Stockings""","""South End Grounds I""",null,103,98,"""BOS""","""BS1""","""BS1"""
1871,"""NA""","""CH1""","""CNA""",null,2,28,null,19,9,null,null,"""N""",null,302,1196,323,52,21,10,60,22,69,21,null,null,241,77,2.76,25,0,1,753,308,6,28,22,229,16,0.829,"""Chicago White Stockings""","""Union Base-Ball Grounds""",null,104,102,"""CHI""","""CH1""","""CH1"""
1871,"""NA""","""CL1""","""CFC""",null,8,29,null,10,19,null,null,"""N""",null,249,1186,328,35,40,7,26,25,18,8,null,null,341,116,4.11,23,0,0,762,346,13,53,34,234,15,0.818,"""Cleveland Forest Citys""","""National Association Grounds""",null,96,100,"""CLE""","""CL1""","""CL1"""
1871,"""NA""","""FW1""","""KEK""",null,7,19,null,7,12,null,null,"""N""",null,137,746,178,19,8,2,33,9,16,4,null,null,243,97,5.17,19,1,0,507,261,5,21,17,163,8,0.803,"""Fort Wayne Kekiongas""","""Hamilton Field""",null,101,107,"""KEK""","""FW1""","""FW1"""
1871,"""NA""","""NY2""","""NNA""",null,5,33,null,16,17,null,null,"""N""",null,302,1404,403,43,21,1,33,15,46,15,null,null,313,121,3.72,32,1,0,879,373,7,42,22,235,14,0.84,"""New York Mutuals""","""Union Grounds (Brooklyn)""",null,90,88,"""NYU""","""NY2""","""NY2"""


In [84]:
teams_prepared = (teams.select(pl.col('teamID'), pl.col('name'), pl.col('yearID'))
                        .rename({'name': 'teamName'})
                 )
teams_prepared.head()

teamID,teamName,yearID
str,str,i64
"""BS1""","""Boston Red Stockings""",1871
"""CH1""","""Chicago White Stockings""",1871
"""CL1""","""Cleveland Forest Citys""",1871
"""FW1""","""Fort Wayne Kekiongas""",1871
"""NY2""","""New York Mutuals""",1871


In [87]:
df_joined = (batting_prepared
             .join(people_prepared, on=pl.col('playerID'), how='left')
             .join(teams_prepared, on=['teamID', 'yearID'], how='left')
             .select(pl.col('yearID'), pl.col('HR'), pl.col('playerName'), pl.col('teamName'))
            )
df_joined

yearID,HR,playerName,teamName
i64,i64,str,str
1920,54,"""Babe Ruth""","""New York Yankees"""
1921,59,"""Babe Ruth""","""New York Yankees"""
1922,42,"""Rogers Hornsby""","""St. Louis Cardinals"""
1922,35,"""Babe Ruth""","""New York Yankees"""
1922,37,"""Tillie Walker""","""Philadelphia Athletics"""
…,…,…,…
2019,35,"""Trevor Story""","""Colorado Rockies"""
2019,49,"""Eugenio Suarez""","""Cincinnati Reds"""
2019,38,"""Gleyber Torres""","""New York Yankees"""
